In [ ]:
from datascience import *
import numpy as np

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In this lecture, I am going to use more interactive plots (they look better) so I am using the plotly.express library.  We won't test you on this but it's good to know.

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Lecture 31

In this lecture, we will explore the use of optimization to find the "best" model and derive least-squares regression.

## Review From Last Lecture

In the last lecture we developed the equations of slope and intercept using the correlation coefficient $r$.

### Standard Units

$$
\text{StandardUnits}(x) = \frac{x - \text{Mean}(x)}{\text{Stdev}(x)} 
$$

In [ ]:
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

### Correlation

$$
\begin{align}
r 
& = \text{Mean}\left(\text{StandardUnits}(x) *  \text{StandardUnits}(y)\right)\\
& = \frac{1}{n} \sum_{i=1}^n \text{StandardUnits}(x_i) *  \text{StandardUnits}(y_i)\\
& = \frac{1}{n}\sum_{i=1}^n \left( \frac{x_i - \text{Mean}(x)}{\text{Stdev}(x)} \right) * \left( \frac{y_i - \text{Mean}(y)}{\text{Stdev}(y)} \right) \\
\end{align}
$$

In [ ]:
def correlation(t, x, y):
    """Computes the correlation between columns x and y"""
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

### Slope and Intercept

$$
\begin{align}
\text{slope} &= r * \frac{\text{Stdev}(y)}{\text{Stdev}(x)}\\
\text{intercept} & = \text{Mean}(y) - \text{slope} * \text{Mean}(x)
\end{align}
$$

In [ ]:
def slope(t, x, y):
    """Computes the slope of the regression line"""
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

In [ ]:
def intercept(t, x, y):
    """Computes the intercept of the regression line"""
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

### Linear Prediction

$$
y_\text{predicted} = \text{slope} * x + \text{intercept}
$$


In [ ]:
def predict_linear(t, x, y):
    """Return an array of the regressions estimates at all the x values"""
    pred_y = slope(t, x, y) * t.column(x) + intercept(t, x, y)
    return pred_y

<br><br><br>

---

## Making Predictions with Linear Regression

We can now compute predictions, but how good are they?  How do we know that we have a good linear fit? To study this we will consider a new dataset.

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.show(5)

In [ ]:
px.scatter(demographics.to_df(), 
           x="College%", 
           y="Median Income",
           color="State")

In [ ]:
correlation(demographics, 'College%', 'Median Income')

**Discussion Question:** Any concerns about the correlation computation being done here?

<br><br><br>



### Making Predictions

Here we will try to predict the income for each district as a function of the percent of college educated people.

In [ ]:
demo_slope = slope(demographics, 'College%', 'Median Income')
demo_intercept = intercept(demographics, 'College%', 'Median Income')
print("Slope:", demo_slope)
print("Intercept:", demo_intercept)

Make the actual predictions.

In [ ]:
demographics = demographics.with_column(
    "Linear Prediction", predict_linear(demographics, 'College%', 'Median Income')
)
demographics

Visualizing the predictions:

In [ ]:
demographics.iscatter('College%', ["Median Income", "Linear Prediction"])

In [ ]:
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income")
xtest = np.arange(0, 75, 1)
fig.add_scatter(x=xtest, 
                y=demo_slope * xtest + demo_intercept,
                name = f"{np.round(demo_slope, 2)} x + {np.round(demo_intercept)}")
fig

<br><br><br>

Is this a good fit? Is it the best fit?

<br><br><br>

---

<center> Return to Slides </center>

---

<br><br><br>

## Computing the Error

The error is the difference between the actual and predicted value:

$$
\text{error} = y - y_\text{predicted}
$$

In a future lecture, we will refer to this error as the **residual**.


In [ ]:
y = demographics.column('Median Income')
predicted = predict_linear(demographics, 'College%', 'Median Income')

errors = y - predicted

In [ ]:
demographics = demographics.with_column('Error', errors)
demographics

<br><br>
What are the districts with the largest error values?

In [ ]:
(demographics
     .with_column("Abs Error", np.abs(demographics.column("Error")))
     .sort("Abs Error", descending=True)
)

What would a large error suggest?

<br><br><br>

### Visualizing the Errors

In [ ]:
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income", hover_name="District")
xtest = np.arange(0, 75, 1)
fig.add_scatter(x=xtest, 
                y=demo_slope * xtest + demo_intercept,
                name = f"{np.round(demo_slope, 2)} x + {np.round(demo_intercept)}")
fig.add_scatter(x=demographics.column("College%").repeat(3), 
                y=np.ravel(np.vstack([y, predicted, np.nan * predicted]).T),
                marker_color="gray", line_width=0.75, name="Errors")
fig

In [ ]:
demographics.ihist('Error', bins=50, rug=True)

<br><br><br>

---
## Summarizing the Overall Error

What is the average error?

In [ ]:
np.mean(errors)

Mean Absolute Error

In [ ]:
np.mean(np.abs(errors))

Mean Squared Error (MSE)

In [ ]:
np.mean(errors ** 2)

Root Mean Squared Error (RMSE)

In [ ]:
np.sqrt(np.mean(errors ** 2))

<br><br>

#### Discussion Question
Assuming $y$ is income in dollars. What are the units of:
1. Mean Absolute Error
2. Mean Squared Error
3. Root Mean Squared Error

<br><br>


<br>

---

## Error as Function of our Model (Line)

In [ ]:
def demographics_rmse(slope, intercept):
    predicted = slope * demographics.column("College%") + intercept 
    actual = demographics.column("Median Income")
    errors = predicted - actual
    rmse = np.sqrt(np.mean(errors ** 2))
    return rmse

The value of our error function for the slope and intercept we derived in last lecture is:

In [ ]:
demographics_rmse(demo_slope, demo_intercept)

What if we used a different slope and intercept value:

In [ ]:
def visualize_demographics_rmse(slope, intercept):
    rmse = demographics_rmse(slope, intercept)
    predicted = slope * demographics.column("College%") + intercept 
    actual = demographics.column("Median Income")
    fig = px.scatter(demographics.to_df(), x="College%", y="Median Income")
    xtest = np.arange(0, 75, 1)
    fig.add_scatter(x=xtest, y=slope * xtest + intercept,
                    name = f"{np.round(slope, 2)} x + {np.round(intercept)}")
    fig.add_scatter(x=demographics.column("College%").repeat(3), 
                    y=np.ravel(np.vstack([actual, predicted, np.nan * predicted]).T),
                    marker_color="gray", line_width=0.75, name="Errors")
    fig.update_layout(title=f"RMSE = {np.round(rmse, 2)}")
    return fig

In [ ]:
visualize_demographics_rmse(demo_slope, demo_intercept)

In [ ]:
visualize_demographics_rmse(demo_slope+1000, demo_intercept - 50000)

<br><br><br>

---

Varying the Slope and Intercept and Plotting the RMSE

In [ ]:
alt_slopes = demo_slope + np.arange(-20, 20)
rmses = []
for new_slope in alt_slopes:
    rmses = np.append(rmses, demographics_rmse(new_slope, demo_intercept))

variations = Table().with_columns("Slope", alt_slopes, "RMSE", rmses)

In [ ]:
fig = px.scatter(variations.to_df(), x="Slope", y="RMSE")
fig.add_scatter(x=[demo_slope], y=[demographics_rmse(demo_slope, demo_intercept)], marker_size=10, 
                name="Best Slope")

What if we tried to change the intercept value while using the best slope so far?

In [ ]:
alt_intercepts = demo_intercept + np.arange(-2000, 2000, 100)
rmses = []
for new_intercept in alt_intercepts:
    rmses = np.append(rmses, demographics_rmse(demo_slope, new_intercept))

variations = Table().with_columns("Intercept", alt_intercepts, "RMSE", rmses)
fig = px.scatter(variations.to_df(), x="Intercept", y="RMSE")
fig.add_scatter(x=[demo_intercept], y=[demographics_rmse(demo_slope, demo_intercept)], marker_size=10, 
                name="Best Intercept")

What if we tried changing both the slope and the intercept at the same time?

In [ ]:
alt_slopes = demo_slope + np.arange(-100, 100, 1)
alt_intercepts = demo_intercept + np.arange(-2000, 2000, 10)
variations = Table(["Slope", "Intercept", "RMSE"])
for new_slope in alt_slopes:
    for new_intercept in alt_intercepts:
        rmse = demographics_rmse(new_slope, new_intercept)
        variations.append([new_slope, new_intercept, rmse])
    
variations
go.Figure(data=[
    go.Contour(x=variations.column("Slope"), y=variations.column("Intercept"), z=variations.column("RMSE")), 
    go.Scatter(x=[demo_slope], y=[demo_intercept], marker_color="red")
],
layout=dict(width = 800,height=600, xaxis_title="Slope", yaxis_title="Intercept"))

In three dimensions:

In [ ]:
go.Figure(data=[
    go.Surface(x = alt_slopes, y = alt_intercepts,
               z=variations.column("RMSE").reshape(len(alt_slopes), len(alt_intercepts)).T),
    go.Scatter3d(x=[demo_slope], y=[demo_intercept], z=[demographics_rmse(demo_slope, demo_intercept)])], 
          layout=dict(height=1000, 
                      scene_xaxis_title="Slope", scene_yaxis_title="Intercept", 
                      scene_zaxis_title="RMSE"))

<br><br><br>

---

<center> Return to Slides </center>

---

<br><br><br>


## Numerical Optimization

If our goal is just to find the parameters of our line that minimize some kind of error, we can use numerical optimization tools.  Suppose we wanted to minimize the function:

$$
f(x) = \left(x - 2\right)^2 + 3
$$

In [ ]:
def f(x):
    return ((x-2)**2) + 3

In [ ]:
x = np.arange(1, 3, 0.1)
y = f(x)
px.line(x=x, y=y)

We could use the minimize function in the `datascience` package. The function returns the arguments to the functions that result in the minimum value:

$$
\texttt{minimize(f)}\, = \, \arg\min_{a,b, \ldots} f(a,b\ldots)
$$

In [ ]:
minimize(f)

In [ ]:
print("x_min =", minimize(f))
print("f(x_min) =", f(minimize(f)))

In [ ]:
fig = px.line(x=x, y=y)
fig.add_scatter(x=[minimize(f)], y=[f(minimize(f))],
                name="Minimum", marker_color="red", marker_size=10)

Minimize works for even more complex functions.

$$
f(x) = 2 * \sin(\pi x) + x^3 + x^4 + \sin(10x)
$$

In [ ]:
def complicated_function(x):
    return 2 * np.sin(x*np.pi) + x ** 3 + x ** 4 + np.sin(x * 10)

In [ ]:
x = np.arange(-1.5, 1.5, 0.01)
y2 = complicated_function(x)
px.line(x=x, y=y2)

We can still use minimize to find the minimum:

In [ ]:
x_min = minimize(complicated_function)
print("x_min =", x_min)
print("f(x_min) =", complicated_function(x_min))

In [ ]:
fig = px.line(x=x, y=y2)
fig.add_scatter(x=[x_min],
                y=[complicated_function(x_min)],
                name="Minimum", marker_color="red", marker_size=10)

We can even minimize multidimensional functions:

$$
\texttt{surface_function(a,b)} = -\frac{\cos\left(\pi \sqrt{(a+0.5)^2 + b^2}\right)}{\sqrt{(a+0.5)^2 + b^2} + 1}
$$

In [ ]:
def surface_function(a, b):
    d = np.sqrt( (a+0.5)**2 + b**2 )
    return -np.cos(np.pi* d) / (d**2 + 1)

In [ ]:
a_min, b_min = minimize(surface_function)
[a_min, b_min]

Visualizing the surface and the minimum:

In [ ]:
xs = np.arange(-1.5, 1.5, 0.01)
ys = np.arange(-1.5, 1.5, 0.01)
x, y = np.meshgrid(xs, ys)
zs = surface_function(x.flatten(), y.flatten())
go.Figure(data=[
    go.Surface(x = xs, y = ys,
               z=zs.reshape(len(xs), len(ys))),
    go.Scatter3d(x=[a_min], y=[b_min], z=[surface_function(a_min, b_min)])
    ], 
    layout=dict(height=1000, 
                scene_xaxis_title="a", scene_yaxis_title="b", 
                scene_zaxis_title="surface"))

<br><br><br>

---

### Minimizing RMSE 

We can use minimize to find the slope and intercept that minimize root mean squared error in our predictions:

In [ ]:
minimize(demographics_rmse)

How does this compare to the slope and intercept we derived earlier?

In [ ]:
[demo_slope, demo_intercept]

What happens if we minimize the mean squared error instead of the root mean squared error?

In [ ]:
def demographics_mse(slope, intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = slope*x + intercept
    return np.mean(((y - estimate) ** 2))

In [ ]:
minimize(demographics_mse)

What if we wanted to use the absolute error instead?

In [ ]:
def demographics_mae(any_slope, any_intercept):
    x = demographics.column('College%')
    y = demographics.column('Median Income')
    estimate = any_slope*x + any_intercept
    return np.mean(np.abs(y - estimate))

In [ ]:
minimize(demographics_mae)

That is different!

In [ ]:
mae_slope, mae_intercept = minimize(demographics_mae)
fig = px.scatter(demographics.to_df(), x="College%", y="Median Income", color="State")
xtest = np.arange(0, 75, 0.1)
fig.add_scatter(x=xtest, 
                y=demo_slope * xtest + demo_intercept,
                name = f"Least Squares: {np.round(demo_slope, 2)} x + {np.round(demo_intercept)}")
fig.add_scatter(x=xtest, 
                y=mae_slope * xtest + mae_intercept,
                name = f"MAE: {np.round(mae_slope, 2)} x + {np.round(mae_intercept)}")
fig

<br><br><br>

---

<center> Return to Slides </center>

---

<br><br><br>

<br><br><br>

---

## Multiple Linear Regression

We can also use multiple variables to help predict a single variable.  

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid.show(5)

In [ ]:
px.scatter_3d(
    hybrid.to_df(), 
    x="mpg", y="acceleration", z="msrp",
    hover_name="vehicle", 
    color="class", 
    height=800
)

Suppose we use the model:

$$
y = a * \text{acc} + b * \text{mpg} + c
$$

In [ ]:
def hybrid_rmse(a, b, c):
    actual = hybrid.column("msrp")
    acc = hybrid.column("acceleration")
    mpg = hybrid.column("mpg")
    predicted = a*acc + b*mpg + c
    mse = np.sqrt(np.mean((actual - predicted)**2))
    return mse

In [ ]:
a, b, c = minimize(hybrid_rmse)
[a, b, c]

In [ ]:
print(f"Error: {hybrid_rmse(a, b, c):,}")

In [ ]:
mpg_range = np.arange(10, 80)
acceleration_range = np.arange(5, 25)
predictions = Table(["mpg", "acc", "pred"])
for mpg in mpg_range:
    for acc in acceleration_range: 
        pred = a * acc + b * mpg + c
        predictions.append([mpg, acc, pred])

In [ ]:
fig = px.scatter_3d(
    hybrid.to_df(), 
    x="mpg", y="acceleration", z="msrp",
    hover_name="vehicle", 
    color="class", 
    height=800
)
fig.add_surface(
    x = mpg_range, y = acceleration_range,
    z = predictions.column("pred").reshape(len(mpg_range), len(acceleration_range)).T,
    showscale=False
)

## Fitting Non-linear Data

We could try to improve our predictions by defining a more complex equation:

$$
\text{y} = a * \text{acc} + b *\text{mpg} + c * \text{acc}^2  + d * \text{mpg}^2 + e 
$$



In [ ]:
def hybrid_better_rmse(a, b, c, d, e):
    actual = hybrid.column("msrp")
    acc = hybrid.column("acceleration")
    mpg = hybrid.column("mpg")
    predicted = a*acc + b*mpg + c*acc**2 + d*mpg**2 + e
    mse = np.sqrt(np.mean((actual - predicted)**2))
    return mse

In [ ]:
a, b, c, d, e = minimize(hybrid_better_rmse)
[a, b, c, d, e]

In [ ]:
print(f"Error: {hybrid_better_rmse(a,b,c,d,e):,}")

In [ ]:
mpg_range = np.arange(10, 80)
acceleration_range = np.arange(5, 25)
predictions = Table(["mpg", "acc", "pred"])
for mpg in mpg_range:
    for acc in acceleration_range: 
        pred = a*acc + b*mpg + c*acc**2 + d*mpg**2 + e
        predictions.append([mpg, acc, pred])
        
fig = px.scatter_3d(
    hybrid.to_df(), 
    x="mpg", y="acceleration", z="msrp",
    hover_name="vehicle", 
    color="class", 
    height=800
)
fig.add_surface(
    x = mpg_range, y = acceleration_range,
    z = predictions.column("pred").reshape(len(mpg_range), len(acceleration_range)).T,
    showscale=False
)